# Predicting Yelp Star Ratings


The objective of this notebook is to predict a new venue's popularity from information available when the venue opens.  We will do this by machine learning from a data set of venue popularities provided by Yelp.  The data set contains meta data about the venue (where it is located, the type of food served, etc.).  It also contains a star rating. The implementation of the prediction scheme is detailed in the following sections.

In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

## Download and parse the incoming data


The training data are a series of JSON objects, in a Gzipped file. Python supports Gzipped files natively: [`gzip.open`](https://docs.python.org/2/library/gzip.html) has the same interface as `open`, but handles `.gz` files automatically.

The built-in json package has a `loads()` function that converts a JSON string into a Python dictionary.  We could call that once for each row of the file. [`ujson`](http://docs.micropython.org/en/latest/library/ujson.html) has the same interface as the built-in `json` library, but is *substantially* faster (at the cost of non-robust handling of malformed json).  We will use that inside a list comprehension to get a list of dictionaries:

In [9]:
import ujson as json
import gzip

with gzip.open('yelp_train_academic_dataset_business.json.gz') as f:
    data = [json.loads(line) for line in f]
data[:2] #venue information is a dictionary

[{'business_id': 'vcNAWiLM4dR7D2nwwJ7nCA',
  'full_address': '4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018',
  'hours': {'Tuesday': {'close': '17:00', 'open': '08:00'},
   'Friday': {'close': '17:00', 'open': '08:00'},
   'Monday': {'close': '17:00', 'open': '08:00'},
   'Wednesday': {'close': '17:00', 'open': '08:00'},
   'Thursday': {'close': '17:00', 'open': '08:00'}},
  'open': True,
  'categories': ['Doctors', 'Health & Medical'],
  'city': 'Phoenix',
  'review_count': 7,
  'name': 'Eric Goldberg, MD',
  'neighborhoods': [],
  'longitude': -111.983758,
  'state': 'AZ',
  'stars': 3.5,
  'latitude': 33.499313,
  'attributes': {'By Appointment Only': True},
  'type': 'business'},
 {'business_id': 'JwUE5GmEO-sH1FuwJgKBlQ',
  'full_address': '6162 US Highway 51\nDe Forest, WI 53532',
  'hours': {},
  'open': True,
  'categories': ['Restaurants'],
  'city': 'De Forest',
  'review_count': 26,
  'name': 'Pine Cone Restaurant',
  'neighborhoods': [],
  'longitude': -89.335844,
  's

In Scikit Learn, the labels to be predicted, in this case, the stars, are always kept in a separate data structure than the features.  Let's get in this habit now, by creating a separate list of the ratings:

In [11]:
star_ratings = [row['stars'] for row in data]
star_ratings[0:2] # ratings is a list of floats per venue

[3.5, 4.0]

## Average Number of Venues per City

The venues belong to different cities.  You can imagine that the ratings in some cities are probably higher than others.  Let's build an estimator to make a prediction based on this, but first we need to work out the average rating for each city.  For this problem, we can create a dictionary where the key is the city name and value is the venue counter per city. The collection modeule's `defaultdict` class defaults values for keys taht haven't been used. Thus,

In [13]:
from collections import defaultdict
star_sum = defaultdict(int)
count = defaultdict(int)

we can increment any key of `stars` or `count` without first worrying whether the key exists.  We need to go through the `data` and `star_ratings` list together, which we can do with the `zip()` function.

In [16]:
for row, stars in zip(data, star_ratings):
    # increment the running sum in star_sum
    # increment the running count in count
    star_sum[row['city']] += stars
    count[row['city']] += 1

Now we can calculate the average ratings.  Again, a dictionary makes a good container.

In [31]:
avg_stars = dict()
for city in star_sum:
    # calculate average star rating and store in avg_stars
    avg_stars[city] = star_sum[city]/count[city]

for key in list(avg_stars)[:5]:
    print (key, avg_stars[key])

Phoenix 3.6702903946388683
De Forest 3.75
Mc Farland 3.1
Middleton 3.611111111111111
Madison 3.6457337883959045


## Predicting the Ratings of a Venue by the City

Now, let's build a custom estimator that will make a prediction based solely on the city of a venue.  It is tempting to hard-code the answers from the previous section into this model, but we're going to resist and do things properly.

This custom estimator will have a `.fit()` method.  It will receive `data` as its argument `X` and `star_ratings` as `y`, and should repeat the calculation of the previous problem there.  Then the `.predict()` method can look up the average rating for the city of each record it receives.

In [32]:
from sklearn import base
from collections import defaultdict

class CityEstimator(base.BaseEstimator, base.RegressorMixin):
    
    def __init__(self):
        self.avg_stars = dict()
    
    def fit(self, X, y):
        star_sum = defaultdict(int)
        count = defaultdict(int)

        # Store the average rating per city in self.avg_stars
        for row, stars in zip(X, y):
            star_sum[row['city']] += stars
            count[row['city']] += 1
        
        for city in star_sum:
            self.avg_stars[city] = star_sum[city]/count[city]

        return self
    
    def predict(self, X):
        result = []
        for row in X:
            if row['city'] not in self.avg_stars: # if the city is not in the training set, we return 0 as the rating
                result.append(0)
            else:
                result.append(self.avg_stars[row['city']])
        return result

Now we can create an instance of our estimator, train it and show a few of the predictions.

In [34]:
city_est = CityEstimator()
city_est.fit(data, star_ratings)
city_est.predict(data[:5])

[3.6702903946388683, 3.75, 3.75, 3.75, 3.75]

## Predicting the Ratings of a Venue by the Lattitude Longitude

You can imagine that a city-based model might not be sufficiently fine-grained. For example, we know that some neighborhoods are trendier than others.  Use the latitude and longitude of a venue as features that help you understand neighborhood dynamics.

Instead of writing a custom estimator, we'll use one of the built-in estimators in Scikit Learn.  Since these estimators won't know what to do with a list of dictionaries, we'll build a `ColumnSelectTransformer` that will return an array containing selected keys of our feature matrix.  While it is tempting to hard-code the latitude and longitude in here, this transformer will be more useful in the future if we write it to work on an arbitrary list of columns.

In [37]:
class ColumnSelectTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, col_names):
        self.col_names = col_names  # We will need these in transform()
    
    def fit(self, X, y=None):
        # This transformer doesn't need to learn anything about the data,
        # so it can just return self without any further processing
        return self
    
    def transform(self, X):
        # Return an array with the same number of rows as X and one
        # column for each in self.col_names
        new_X = []
        return [[row[name] for name in self.col_names] for row in X]

Let's test it on a single row, just as a sanity check:

In [38]:
cst = ColumnSelectTransformer(['latitude', 'longitude'])
print(cst.fit_transform(data[:1]))
print(data[:1])
assert (cst.fit_transform(data[:1])
        == [[data[0]['latitude'], data[0]['longitude']]])

[[33.499313, -111.983758]]
[{'business_id': 'vcNAWiLM4dR7D2nwwJ7nCA', 'full_address': '4840 E Indian School Rd\nSte 101\nPhoenix, AZ 85018', 'hours': {'Tuesday': {'close': '17:00', 'open': '08:00'}, 'Friday': {'close': '17:00', 'open': '08:00'}, 'Monday': {'close': '17:00', 'open': '08:00'}, 'Wednesday': {'close': '17:00', 'open': '08:00'}, 'Thursday': {'close': '17:00', 'open': '08:00'}}, 'open': True, 'categories': ['Doctors', 'Health & Medical'], 'city': 'Phoenix', 'review_count': 7, 'name': 'Eric Goldberg, MD', 'neighborhoods': [], 'longitude': -111.983758, 'state': 'AZ', 'stars': 3.5, 'latitude': 33.499313, 'attributes': {'By Appointment Only': True}, 'type': 'business'}]


Now, let's feed the output of the transformer in to a `sklearn.neighbors.KNeighborsRegressor`.  As a sanity check, we'll test it with the first 5 rows.  To truly judge the performance, we'd need to make a test/train split.

In [39]:
from sklearn.neighbors import KNeighborsRegressor

data_transform = cst.fit_transform(data)
knn = KNeighborsRegressor()
knn.fit(data_transform, star_ratings)
test_data = data[:5]
test_data_transform = cst.transform(test_data)
knn.predict(test_data_transform)

array([4. , 4.2, 4. , 3.8, 4.2])

Instead of doing this by hand, let's make a pipeline that takes a list of (name, transformer-or-estimator) tuples. 

In [40]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('transformer', ColumnSelectTransformer(['latitude','longitude'])), 
    ('estimator', knn)])

This should work the same way.

In [41]:
pipe.fit(data, star_ratings)
pipe.predict(test_data)

array([4. , 4.2, 4. , 3.8, 4.2])

The `KNeighborsRegressor` takes the `n_neighbors` hyperparameter, which tells it how many nearest neighbors to average together when making a prediction.  There is no reason to believe that 5 is the optimum value.  Determine a better value of this hyperparameter.   There are several ways to do this:

1. Use [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#sklearn.model_selection.train_test_split) to split your data into a training set and a test set.  Score the performance on the test set.  After finding the best hyperparameter, retrain the model on the full data at that hyperparameter value.

2. Use [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score) to return cross-validation scores on your data for various values of the hyperparameter.  Choose the best one, and retrain the model on the full data.

3. Use [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) to do the splitting, training, and grading automatically.  `GridSearchCV` takes an estimator and acts as an estimator.  You can either give it the `KNeighborsRegressor` directly and put it in a pipeline, or you can pass the whole pipeline into the `GridSearchCV`.  In the latter case, the hyperparameter `param` of an estimator named `est` in a pipeline becomes a hyperparameter of the pipeline with name `est__param`.

Let's use GridSearch to perform both cross validation and find the right values for the hyperparameters of our model,

In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

#X_train, X_test, y_train, y_test = train_test_split(data, star_ratings, test_size=0.3)

param_grid = {'estimator__n_neighbors': [60] }
lat_long_est = GridSearchCV(pipe, param_grid, cv=5)
lat_long_est.fit(data, star_ratings)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('transformer', ColumnSelectTransformer(col_names=['latitude', 'longitude'])), ('estimator', KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform'))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'estimator__n_neighbors': [60]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [45]:
lat_long_est.best_params_ #we can look up from the best_params_ dictionary the optimum number of clusters

{'estimator__n_neighbors': 60}

## Predicting the Ratings of a Venue by the Category

While location is important, we could also try seeing how predictive the
venue's category is.  Lets's build an estimator that considers only the categories.

The categories come as a list of strings, but the built-in estimators all need numeric input.  The standard way to deal with categorical features is **one-hot encoding**, also known as dummy variables.  In this approach, each category gets its own column in the feature matrix.  If the row has a given category, that column gets filled with a 1.  Otherwise, it is 0.

The `ColumnSelectTransformer` from the previous section can be used to extract the categories column as a list of strings.  scikit learn provides [`DictVectorizer`](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html#sklearn.feature_extraction.DictVectorizer), which takes in a list of dictionaries.  It creates a column in the output matrix for each key in the dictionary and fills it with the value associated with it.  Missing keys are filled with zeros.  Therefore, we need to only build a transformer that takes a list of strings to a dictionary with keys given by those strings and values of one.

In [53]:
class DictEncoder(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # X will come in as a list of lists of lists.  Return a list of
        # dictionaries corresponding to those inner lists.
        return [{key:1 for key in double_list[0]} for double_list in X]

That should allow this to pass:

In [51]:
assert (DictEncoder().fit_transform([[['a']], [['b', 'c']]])
        == [{'a': 1}, {'b': 1, 'c': 1}])

Set up a pipeline with your `ColumnSelectTransformer`, your `DictEncoder`, the `DictVectorizer`, and a regularized linear model, like `Ridge`, as the estimator.  This model will have a large number of features, one for each category, so there is a significant danger of overfitting.  Use cross validation to choose the best regularization parameter.

In [54]:
from sklearn.linear_model import Ridge
from sklearn.feature_extraction import DictVectorizer

clf = Ridge()
cat_pipe = Pipeline([
    ('cst', ColumnSelectTransformer(['categories'])),
    ('denc', DictEncoder()),
    ('dvec', DictVectorizer()),
    ('estimator', clf)
])


param_grid = {'estimator__alpha': [1, 10, 100 ,1000] }
category_est = GridSearchCV(cat_pipe, param_grid, cv=5)
category_est.fit(data, star_ratings)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('cst', ColumnSelectTransformer(col_names=['categories'])), ('denc', DictEncoder()), ('dvec', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)), ('estimator', Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'estimator__alpha': [1, 10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [55]:
category_est.best_params_ #we can look up from the best_params_ dictionary the optimum number of alpha

{'estimator__alpha': 10}

## Predicting the Ratings of a Venue by the Attribute

There is even more information in the attributes for each venue.  Let's build an estimator based on these.

Venues attributes may be nested:
```
{
  'Attire': 'casual',
  'Accepts Credit Cards': True,
  'Ambiance': {'casual': False, 'classy': False}
}
```
We wish to encode them with one-hot encoding.  The `DictVectorizer` can do this, but only once we've flattened the dictionary to a single level, like so:
```
{
  'Attire_casual' : 1,
  'Accepts Credit Cards': 1,
  'Ambiance_casual': 0,
  'Ambiance_classy': 0
}
```

Build a custom transformer that flattens the attributes dictionary.  Place this in a pipeline with a `DictVectorizer` and a regressor.

It might be difficult to find a single regressor that does well enough.  A common solution is to use a linear model to fit the linear part of some data, and use a non-linear model to fit the residual that the linear model can't fit.  It can be done as follows,

In [56]:
import collections

class dictFlattener(base.BaseEstimator, base.TransformerMixin):
    
    def fit(self, X, y=None):
        return self
    
    def transform(self,X):
        '''This transformer flattens the 
        attributes dictionary.
        '''
        all_list = []
        for lst in X:
            flattened = {}
            for d in lst:
                for k, v in d.items():
                    if type(v) == str:
                        flattened[k+'_'+v] = 1
                    elif type(v) == bool:
                        flattened[k] = int(v)
                    elif type(v) == int:
                        flattened[k+'_'+str(v)] = 1
                    else:
                        for k_, v_ in v.items():
                            flattened[k+'_'+k_] = int(v_)
            all_list.append(flattened)
    
        return all_list

In [59]:
class doubleLayerModel(base.BaseEstimator, base.TransformerMixin):
    '''This class builds a linear model first and uses a non-linear
    model to fit the residual that the linear model can't fit.
    '''
    def __init__(self, base, residual):
        self.base = base
        self.residual = residual
    
    def fit(self, X, y):
        self.base.fit(X,y)
        self.residual.fit(X, y-self.base.predict(X))
        return self
    
    def predict(self, X):
        return self.base.predict(X) + self.residual.predict(X)

In [60]:
from sklearn.ensemble import RandomForestRegressor

lin_est = Ridge()
non_lin_est = RandomForestRegressor(n_estimators=150, max_depth=15, random_state=0)

attribute_est = Pipeline([
    ('cst', ColumnSelectTransformer(['attributes'])),
    ('dF', dictFlattener()),
    ('dvec', DictVectorizer()),
    ('estimator', doubleLayerModel(lin_est,non_lin_est))
])
attribute_est.fit(data, star_ratings)


Pipeline(memory=None,
     steps=[('cst', ColumnSelectTransformer(col_names=['attributes'])), ('dF', dictFlattener()), ('dvec', DictVectorizer(dtype=<class 'numpy.float64'>, separator='=', sort=True,
        sparse=True)), ('estimator', doubleLayerModel(base=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   ...mators=150, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)))])

## Full Model

So far we have only built models based on individual features.  Now we will build an ensemble regressor that averages together the estimates of the four previous regressors.

In order to use the existing models as input to an estimator, we will have to turn them into transformers.  (A pipeline can contain at most a single estimator.)  We will build a custom `EstimatorTransformer` class that takes an estimator as an argument.  When `fit()` is called, the estimator should be fit.  When `transform()` is called, the estimator's `predict()` method should be called, and its results returned.

In [62]:
import numpy as np
class EstimatorTransformer(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, estimator):
        # What needs to be done here?
        self.estimator = estimator
        
    def fit(self, X, y):
        # Fit the stored estimator.
        # Question: what should be returned?
        self.estimator.fit(X, y)
        
        return self
        
    def transform(self, X):
        # Use predict on the stored estimator as a "transformation".
        # Be sure to return a 2-D array.
        return np.array(self.estimator.predict(X)).reshape(-1, 1)

This should work as follows:

In [63]:
city_trans = EstimatorTransformer(city_est)
city_trans.fit(data, star_ratings)

assert ([r[0] for r in city_trans.transform(data[:5])] == city_est.predict(data[:5]))

We can create an instance of the `EstimatorTransformer` for each of the previous four models. Combine these together in a single feature matrix with a
[`FeatureUnion`](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion).

In [64]:
from sklearn.pipeline import FeatureUnion

lat_long_trans = EstimatorTransformer(lat_long_est)
category_est_tans = EstimatorTransformer(category_est)
attribute_est_trans = EstimatorTransformer(attribute_est)


union = FeatureUnion([('city_trans', city_trans),
                      ('lat_long_trans', lat_long_trans),
                      ('category_est_tans', category_est_tans), 
                      ('attribute_est_trans', attribute_est_trans)])
        # FeatureUnions use the same syntax as Pipelines
    

This should return a feature matrix with four columns.

In [65]:
union.fit(data, star_ratings)
trans_data = union.transform(data[:10])
assert trans_data.shape == (10, 4)

Finally, we can use a pipeline to combine the feature union with a linear regression (or another model) to weight the predictions.

In [66]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
full_est = Pipeline([('featureUnion',union),
                    ('linReg', lin_reg)])
full_est.fit(data, star_ratings)

Pipeline(memory=None,
     steps=[('featureUnion', FeatureUnion(n_jobs=None,
       transformer_list=[('city_trans', EstimatorTransformer(estimator=CityEstimator())), ('lat_long_trans', EstimatorTransformer(estimator=GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('trans...'linReg', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False))])